# Distributional Properties of $\gamma(C)$
## Marseille, June 14: Notebook for Lecture 2

In [ ]:
#import Pkg; Pkg.add("StatsPlots")
using Plots
using LinearAlgebra
using StatsPlots
using Statistics

# Part 2.A: Sample Correlation and its Finite Sample Properties

Suppose that $(X_i,Y_i)$ are iid with finite fourth moment, and denote $\rho=\mathrm{corr}(X,Y)$ their correlation coefficient.

Empirical correlation

$$ \hat{\rho}= 
\frac{ 
    \widehat{\mathrm{cov}}(X,Y)
    }{ 
    \sqrt{\widehat{\mathrm{var}}(X)\widehat{\mathrm{var}}(Y)}
    },
$$

is such that
$$ \sqrt{T}(\hat{\rho}-\rho) \overset{d}{\rightarrow} N(0,[1-\rho^2]^2)\qquad \text{as}\quad T\rightarrow \infty. $$
## Simulate a single correlation (Julia code)

In [ ]:
ρ = 0.5
Z  = randn(100,1)
X  = randn(100,1)
Y  = ρ.*X +sqrt(1-ρ^2).*Z
cor(X,Y)

## Simulate $N$ Sample Correlations

In [ ]:
function ρhat(ρ,T)
    X  = randn(T)
    Y  = ρ.*X +sqrt(1-ρ^2).*randn(T)
    return cor(X,Y)
end
function ρhats(ρ,T,N)
    r = [ρhat(ρ,T) for i in 1:N]
end    
function t_ρ(ρ,T)
    ρ̂ = ρhat(ρ,T)
    return sqrt(T)*(ρ̂-ρ)/(1-ρ̂^2)
end
function tρCDF(ρ,T,N,c)
    δ = [ (t_ρ(ρ,T)<c) for i in 1:N]
    return mean(δ)
end    
    
T = 40
ρ = 0.9
N = 100000
LeftTailρ  = tρCDF(ρ,T,N,-1.64)
RightTailρ = 1- tρCDF(ρ,T,N,1.64)
(LeftTailρ,RightTailρ,LeftTailρ+RightTailρ)

In [ ]:
# Fisher tranformation
function Fisher(r)
    log((1+r)/(1-r))/2
end
function t_ϕ(ρ,T)
    ρ̂ = ρhat(ρ,T)
    return sqrt(T)*(Fisher(ρ̂)-Fisher(ρ))
end

function tϕCDF(ρ,T,N,c)
    δ = [ (t_ϕ(ρ,T)<c) for i in 1:N]
    return mean(δ)
end    
LeftTailϕ  = tϕCDF(ρ,T,N,-1.64)
RightTailϕ = 1- tϕCDF(ρ,T,N,1.64)
(LeftTailϕ,RightTailϕ,LeftTailϕ+RightTailϕ)

## Plot Empirical distribution of $\hat\rho$

In [ ]:
plot(title = "Distribution of ρ̂. (ρ = $ρ, T = $T)", legend=:none)
r = ρhats(ρ,T,N)
density!(r)

In [ ]:
using Distributions
plot!(Normal(ρ,√((1-ρ^2)^2/T)), fill=(0, 0.1,:red),xlims=(0.75,1.05))

## Empirical distribution of $\hat\phi$

In [ ]:
ϕ₀ = Fisher(ρ)
ϕ = [Fisher(r[i]) for i in 1:N] 
plot(title = "Distribution of ϕ̂. (ρ = $ρ, T = $T)", legend=:none,xlims=(0.9,2.1))
density!(ϕ)
#plot!(Normal(ϕ₀ +ρ/(2*T),√(1/T+(6-ρ^2)/(2*T^2))), fill=(0, 0.1,:red))
plot!(Normal(ϕ₀,√(1/T)), fill=(0, 0.1,:red))

# Part 2.B New Parametrization (Case $n=3$)

In [ ]:
#vecl operator
function vecl(A::AbstractMatrix{T}) where T # vectorization operator (below diagonal elements)
    m = LinearAlgebra.checksquare(A)
    v = Vector{T}(undef, (m*(m-1))>>1)
    k = 0
    for j = 1:m-1, i = j+1:m
        @inbounds v[k += 1] = A[i,j]
    end
    return v
end

We will simulate data from the following correlation matrix:

In [ ]:
C = [ 1.0 0.8 0.95
      0.8 1.0 0.7
      0.95 0.7 1.0 ]

Simulate T random variables from N(0,C), and compute empirical correlation matrix

In [ ]:
T = 40; n = size(C,1)
X = randn(T,n)*C^(0.5) # Simulate T vectors from N(0,C)
Ĉ = cor(X)             # Empirical correlation matrix

Let us do this N times and investigate distributional properties

In [ ]:
N = 100000                 # Number of simulations 
ρ̂ = zeros(N,(n*(n-1))>>1)
γ̂ = zeros(N,(n*(n-1))>>1)  
for i ∈ 1:N
    X      = randn(T,n)*C^(0.5)
    Ĉ      = cor(X)
    ρ̂[i,:] = vecl(Ĉ)'
    γ̂[i,:] = vecl(log(Ĉ))'   # New representation
end
mean(ρ̂, dims=1)  # sample average of correlations

What do the sample distributions look like?

In [ ]:
plot(legend=:topleft, xlims=(0.5,1.0))
    density!(ρ̂[:,1],  label = "ρ̂₁")
    density!(ρ̂[:,2], label = "ρ̂₂")
    density!(ρ̂[:,3], label = "ρ̂₃")     

## "Studentize" with sqrt(avar(ρ) = (1-ρ²)^2)

In [ ]:
ρ = vecl(C)                # vector of correlations
plot(legend=:topleft,xlims=(-4,4),title="'Infeasible' t-stat") 
    density!( √T*(ρ̂[:,1].-ρ[1])/(1-ρ[1]^2),  label = "T(ρ̂₁)")
    density!( √T*(ρ̂[:,2].-ρ[2])/(1-ρ[2]^2), label = "T(ρ̂₂)")
    density!( √T*(ρ̂[:,3].-ρ[3])/(1-ρ[3]^2), label = "T(ρ̂₃)")
    plot!(x -> exp(-x^2/2)/√(2π), linecolor=:black, linestyle=:dash, label = "N(0,1)")

In [ ]:
plot(legend=:topleft,xlims=(-4,4),title="t-statistics") 
    density!( √T*(ρ̂[:,1].-ρ[1])./(1 .- ρ̂[:,1].^2),  label = "t(ρ̂₁)")
    density!( √T*(ρ̂[:,2].-ρ[2])./(1 .- ρ̂[:,2].^2), label = "t(ρ̂₂)")
    density!( √T*(ρ̂[:,3].-ρ[3])./(1 .- ρ̂[:,3].^2), label = "t(ρ̂₃)")
    plot!(x -> exp(-x^2/2)/√(2π), linecolor=:black, linestyle=:dash, label = "N(0,1)")

## Fisher transformed correlations look much better

In [ ]:
ϕ = atanh.(ρ)                # vector of Fisher transformed correlations
ϕ̂ = atanh.(ρ̂)                # Simulated F-transformed correlations
plot(legend=:topleft) 
    density!( √T*(ϕ̂[:,1].-ϕ[1]), label = "t-ϕ̂₁")
    density!( √T*(ϕ̂[:,2].-ϕ[2]), label = "t-ϕ̂₂")
    density!( √T*(ϕ̂[:,3].-ϕ[3]), label = "t-ϕ̂₃")
    plot!(x -> exp(-x^2/2)/√(2π), linecolor=:black, linestyle=:dash, label = "N(0,1)")

## Generalized Fisher Transformed correlations. [Can be improved using avar(γ̂)]

In [ ]:
γ = vecl(log(C))             # GFT correlation matrix
plot(legend=:topleft) 
    density!( √T*(γ̂[:,1].-γ[1]), label = "t-γ̂₁")
    density!( √T*(γ̂[:,2].-γ[2]), label = "t-γ̂₂")
    density!( √T*(γ̂[:,3].-γ[3]), label = "t-γ̂₃")
    plot!(x -> exp(-x^2/2)/√(2π), linecolor=:black, linestyle=:dash, label = "N(0,1)")

## Inter-dependence reveal advantage of GFT

In [ ]:
corrplot(ρ̂[1:5000,:], title = "Empirical Correlations: ρ̂")

In [ ]:
corrplot(ϕ̂[1:5000,:], title = "Empirical Fisher-transformed Correlations: ϕ̂")    

In [ ]:
corrplot(γ̂[1:5000,:], title = "New Pametrization: γ̂")#, aspect_ratio=1)

In [ ]:
cornerplot(γ̂[1:5000,:])

In [ ]:
cornerplot(γ̂[1:5000,:], markercolor =:blues, aspect_ratio=1)